In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, LeaveOneOut, KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [2]:

data = pd.read_pickle('../create_SL_data/data_6months.pkl')

data

,patient_number,cluster_Basophils_size-diff%_pre_to_3h,cluster_Bcells_size-diff%_pre_to_3h,cluster_Doublets_size-diff%_pre_to_3h,cluster_Eosinophils_size-diff%_pre_to_3h,cluster_HSPCs_size-diff%_pre_to_3h,cluster_LinNeg_size-diff%_pre_to_3h,cluster_Monocytes_size-diff%_pre_to_3h,cluster_NKcells_size-diff%_pre_to_3h,cluster_NeutrophilCD16neg_size-diff%_pre_to_3h,...,q90_difference_pre_7d_cluster_NeutrophilsCD25_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_NeutrophilsHLA_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_PreNeu_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_PreNeu1_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_PreNeu2_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_PreNeu3_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_Tc_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_Th_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_pDC_158Gd_pSTAT3_Y705,CHR
0,01,35.135135,60.493827,32.682927,34.866434,90.239574,1120.000000,35.968601,57.288136,50.604844,...,0.000000,0.173798,0.228703,-0.634862,0.520689,0.445879,0.067349,0.119710,0.528887,1
1,02,-30.165160,-15.625000,-13.513514,-45.743146,-39.000000,223.109244,-37.692571,-11.141304,-39.835804,...,0.000000,0.195533,0.319469,-0.824650,0.814652,0.649210,0.225261,0.701564,0.523819,1
2,03,91.287290,60.059172,98.181818,25.997138,154.830918,295.501022,62.084592,-3.809524,40.061262,...,0.450780,0.013442,-0.006067,-2.014077,0.234621,0.509549,0.399494,0.270279,0.256670,1
3,04,41.385171,88.732394,13.978495,24.392621,40.764331,97.575758,6.520173,25.301205,18.461538,...,0.125471,0.107060,-0.104685,-1.469164,0.578006,0.312356,0.016451,-0.092338,0.236907,1
4,05,149.305556,196.078431,-3.225806,6.167401,30.000000,150.666667,98.670465,127.884615,22.222222,...,0.414612,0.588062,0.377221,-0.530531,0.544317,0.000000,0.104006,0.037562,0.000000,1
5,06,7.061267,30.208333,46.153846,-26.833013,55.555556,110.256410,-16.031746,21.428571,10.078111,...,0.805617,0.126909,0.144516,-1.011029,0.615345,0.184870,0.058221,-0.048725,0.000000,0
6,07,-7.244374,-0.273224,10.000000,6.280428,-31.139489,368.000000,0.289436,26.732673,22.421163,...,-0.723297,0.101798,0.123791,-0.386960,0.122739,0.275844,0.265686,0.621109,-0.836546,1
7,08,-3.514676,21.551724,38.461538,-29.945629,46.462715,1622.844828,3.701299,-25.128205,-10.869139,...,0.751987,0.155500,0.112758,-1.508415,-0.032117,0.243146,0.096635,-0.365599,0.494612,1
8,09,-21.217515,-32.635983,-71.604938,-5.039834,-32.792793,182.887701,-47.490668,-53.794940,4.181891,...,1.708393,0.515315,0.439256,-1.008428,0.613103,-0.129841,0.306030,0.615110,0.539467,1
9,11,13.279730,20.719424,-11.538462,-10.527406,60.229885,264.285714,-9.048117,37.896825,7.837426,...,0.000000,0.329252,0.167228,-0.809707,0.860805,-0.124076,0.075963,-0.064593,-0.068443,1


In [3]:
data['patient_number'].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '11', '12',
       '13', '14', '15', '16', '17', '18', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '32', '33', '34', '35', '36', '37',
       '38'], dtype=object)

In [4]:
X = data.drop(['CHR', 'patient_number'], axis=1)  
y = data['CHR']                                  

patient_numbers = data['patient_number']

In [10]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import LeaveOneOut, GridSearchCV
from sklearn.metrics import f1_score, accuracy_score

models = {
    'RandomForest': {
        'model': RandomForestClassifier(random_state=42),
        'params': {'n_estimators': [50, 100, 200, 300, 500], 'max_depth': [3, 5, 10, 50], 'max_features' : ['sqrt', None]}

    },
    'KNN': {
        'model': KNeighborsClassifier(),
        'params': {'n_neighbors': [3, 5, 7, 10], 'weights': ['uniform', 'distance']}
    },
    'LogisticRegression': {
        'model': LogisticRegression(random_state=42, max_iter=1000),
        'params': {'C': [0.1, 1, 10], 'solver': ['liblinear', 'lbfgs']}
    },
    'LDA': {
        'model': LinearDiscriminantAnalysis(),
        'params': {'solver': ['svd', 'lsqr']}
    }
}

feature_importances = {name: pd.DataFrame(np.zeros((X.shape[0], X.shape[1])), columns=X.columns) for name in models if name == 'RandomForest'}
scores = {name: {'f1_scores': [], 'accuracy_scores': [], 'best_params': []} for name in models}

outer_cv = LeaveOneOut()
inner_cv = LeaveOneOut()

for fold_idx, (train_idx, test_idx) in enumerate(outer_cv.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    for name, config in models.items():
        grid_search = GridSearchCV(config['model'], config['params'], cv=inner_cv, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        
        y_pred = best_model.predict(X_test)
        f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
        accuracy = accuracy_score(y_test, y_pred)

        scores[name]['f1_scores'].append(f1)
        scores[name]['accuracy_scores'].append(accuracy)
        scores[name]['best_params'].append(grid_search.best_params_)

        if name == 'RandomForest':
            feature_importances[name].iloc[fold_idx, :] = best_model.feature_importances_
        
    print('fold', fold_idx)

results = {}
for name in models:
    average_f1 = np.mean(scores[name]['f1_scores'])
    average_accuracy = np.mean(scores[name]['accuracy_scores'])
    results[name] = {'Average F1 Score': average_f1, 'Average Accuracy': average_accuracy, 'Best Parameters': scores[name]['best_params'][-1]}

    if name == 'RandomForest':
        mean_fi = feature_importances[name].mean(axis=0).sort_values(ascending=False)
        results[name]['Feature Importances'] = mean_fi

for name, result in results.items():
    print(f"{name} Results:")
    for key, value in result.items():
        print(f"{key}: {value}")


/Home/siv32/fiv012/.conda/envs/master/lib/python3.10/site-packages/sklearn/covariance/_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
/Home/siv32/fiv012/.conda/envs/master/lib/python3.10/site-packages/sklearn/covariance/_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(


fold 0


/Home/siv32/fiv012/.conda/envs/master/lib/python3.10/site-packages/sklearn/covariance/_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
/Home/siv32/fiv012/.conda/envs/master/lib/python3.10/site-packages/sklearn/covariance/_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(


fold 1


/Home/siv32/fiv012/.conda/envs/master/lib/python3.10/site-packages/sklearn/covariance/_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
/Home/siv32/fiv012/.conda/envs/master/lib/python3.10/site-packages/sklearn/covariance/_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(


...

fold 31


/Home/siv32/fiv012/.conda/envs/master/lib/python3.10/site-packages/sklearn/covariance/_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
/Home/siv32/fiv012/.conda/envs/master/lib/python3.10/site-packages/sklearn/covariance/_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(


fold 32


/Home/siv32/fiv012/.conda/envs/master/lib/python3.10/site-packages/sklearn/covariance/_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
/Home/siv32/fiv012/.conda/envs/master/lib/python3.10/site-packages/sklearn/covariance/_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(


fold 33
RandomForest Results:
Average F1 Score: 0.9411764705882353
Average Accuracy: 0.9411764705882353
Best Parameters: {'max_depth': 3, 'max_features': 'sqrt', 'n_estimators': 50}
Feature Importances: 90th_142Nd_cCaspase3_3h_cluster_NeutrophilCD16neg           0.042043
q90_difference_pre_3h_cluster_Doublets_176Yb_pS6_S240244    0.040158
90th_142Nd_cCaspase3_3h_cluster_Doublets                    0.027083
90th_151Eu_pSTAT3 S727_pre_cluster_Doublets                 0.026095
90th_150Nd_pSTAT5 Y694_pre_cluster_PreNeu2                  0.023749
                                                              ...   
90th_165Ho_pCREB S133_3h_cluster_PreNeu3                    0.000000
90th_165Ho_pCREB S133_3h_cluster_PreNeu2                    0.000000
90th_165Ho_pCREB S133_3h_cluster_PreNeu1                    0.000000
90th_165Ho_pCREB S133_3h_cluster_PreNeu                     0.000000
q90_difference_pre_7d_cluster_pDC_158Gd_pSTAT3_Y705         0.000000
Length: 1463, dtype: float64
KNN Resul

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [6]:
model1 = RandomForestClassifier(n_estimators=50, max_depth=3, max_features='sqrt', random_state=42)
model2 = KNeighborsClassifier(n_neighbors=3, weights='uniform')
model3 = LinearDiscriminantAnalysis(solver='svd')

model1.fit(X, y)
model2.fit(X, y)
model3.fit(X, y)

LinearDiscriminantAnalysis()

In [7]:
model1.predict(X)

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1])

In [10]:
model2.predict(X)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [11]:
model3.predict(X)

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [10]:
model = RandomForestClassifier(n_estimators=50, max_depth=3, max_features='sqrt', random_state=42)
model.fit(X, y)

RandomForestClassifier(max_depth=3, n_estimators=50, random_state=42)

In [15]:
pd.set_option('display.max_colwidth', None)

coefficients = model.feature_importances_  

feature_importance = pd.DataFrame(data={'Feature': X.columns, 'Importance': coefficients})

feature_importance['Absolute Importance'] = feature_importance['Importance'].abs()
feature_importance = feature_importance.sort_values(by='Absolute Importance', ascending=False)

feature_importance.head(30)

,Feature,Importance,Absolute Importance
1071,q90_difference_pre_3h_cluster_NKcells_142Nd_cCaspase3,0.066667,0.066667
331,90th_142Nd_cCaspase3_3h_cluster_NeutrophilCD16neg,0.038351,0.038351
543,90th_171Yb_pERK T202Y204_3h_cluster_NeutrophilsHLA,0.036574,0.036574
325,90th_142Nd_cCaspase3_3h_cluster_Doublets,0.028315,0.028315
500,90th_165Ho_pCREB S133_3h_cluster_Monocytes,0.022222,0.022222
574,90th_173Yb_STAT3tot_3h_cluster_HSPCs,0.022222,0.022222
630,90th_143Nd_pCRKL Y207_7d_cluster_Eosinophils,0.022222,0.022222
477,90th_159Tb_pMAPKAPK T334_3h_cluster_Doublets,0.022222,0.022222
569,90th_172Yb_pS6 S235S236_3h_cluster_pDC,0.022222,0.022222
619,90th_142Nd_cCaspase3_7d_cluster_NeutrophilsHLA,0.022222,0.022222


In [15]:
model.score(X, y)

1.0

In [5]:
data_pool2 = pd.read_pickle('../create_SL_data_pool2/data_6months_an_pool2.pkl')
data_pool2

,patient_number,cluster_Basophils_size-diff%_pre_to_3h,cluster_Bcells_size-diff%_pre_to_3h,cluster_Doublets_size-diff%_pre_to_3h,cluster_Eosinophils_size-diff%_pre_to_3h,cluster_HSPCs_size-diff%_pre_to_3h,cluster_LinNeg_size-diff%_pre_to_3h,cluster_Monocytes_size-diff%_pre_to_3h,cluster_NKcells_size-diff%_pre_to_3h,cluster_NeutrophilCD16neg_size-diff%_pre_to_3h,...,q90_difference_pre_7d_cluster_NeutrophilsCD25_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_NeutrophilsHLA_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_PreNeu_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_PreNeu1_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_PreNeu2_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_PreNeu3_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_Tc_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_Th_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_pDC_158Gd_pSTAT3_Y705,CHR
0,39,-18.402528,18.560953,-32.697947,-37.497949,-31.250000,-22.003284,-22.095407,32.439177,-10.320844,...,0.825204,-0.005506,-0.227297,1.102690,0.450017,-0.222597,-0.230182,-0.094639,-0.220637,1
1,40,20.930233,17.784615,-13.824420,-0.543613,43.076923,26.111111,-0.762943,-0.108108,95.084522,...,-2.051880,0.258281,0.224172,-0.495938,0.204924,-0.015509,0.058554,0.131103,0.171567,0
2,41,9.806781,20.764667,176.031861,-5.221441,18.918919,23.404255,3.371212,10.900474,-14.087567,...,-0.509376,-0.052363,0.187480,-0.102902,0.313390,0.022359,0.107179,0.150815,-0.086913,1
3,42,89.566360,60.869565,47.547974,68.057971,198.809524,54.285714,88.723214,51.600753,41.558257,...,0.391471,0.547319,0.078384,-0.290018,0.142345,0.050058,-0.041273,0.023032,0.288826,0
4,44,-14.055164,4.975124,2.919708,-16.802940,-10.630758,-15.873016,-14.603537,-25.163827,-24.648171,...,0.565738,0.269533,0.623075,-0.729533,0.262059,0.160757,0.031653,0.489867,0.000000,1
5,45,-79.077006,-82.129278,-55.022573,-76.265173,-77.248677,-60.127932,-78.236521,-75.521558,-80.491035,...,-1.050872,-0.732582,0.300580,0.885093,0.381198,-0.058334,0.075121,0.074394,0.346823,1
6,54,63.366337,183.458647,209.836066,18.304779,80.000000,74.000000,80.403458,47.356322,171.248724,...,-0.428832,0.165721,-0.057874,-0.548290,0.360393,-0.466679,-0.103077,-0.119683,-0.479654,0
7,56,11.864407,0.000000,3.225806,8.862705,82.500000,6.153846,2.743142,-26.373626,9.806673,...,-0.059720,1.024588,-0.965296,-0.636475,0.371345,0.000000,0.000000,0.000000,0.000000,1
8,57,100.847212,67.123288,100.000000,138.268004,147.321429,101.666667,160.173997,46.296296,108.943249,...,0.000000,0.000000,0.467656,-0.591875,0.227060,0.000000,-0.004091,-0.015839,0.000000,1


In [7]:
X_test = data_pool2.drop(['CHR', 'patient_number'], axis=1)  
y_test = data_pool2['CHR']  

y_test    

0    1
1    0
2    1
3    0
4    1
5    1
6    0
7    1
8    1
Name: CHR, dtype: int64

In [11]:
pred = model.predict(X_test)

In [12]:
pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1])